# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [1]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [2]:
import homework2

['./nsynth_subset/guitar_acoustic_021-101-050.wav', './nsynth_subset/guitar_acoustic_015-077-050.wav', './nsynth_subset/guitar_acoustic_014-054-075.wav', './nsynth_subset/vocal_synthetic_003-024-100.wav', './nsynth_subset/guitar_acoustic_021-085-100.wav', './nsynth_subset/vocal_synthetic_003-046-075.wav', './nsynth_subset/guitar_acoustic_010-066-127.wav', './nsynth_subset/guitar_electronic_022-083-100.wav', './nsynth_subset/guitar_acoustic_010-073-127.wav', './nsynth_subset/guitar_electronic_022-077-075.wav', './nsynth_subset/guitar_acoustic_010-083-127.wav', './nsynth_subset/vocal_acoustic_000-053-075.wav', './nsynth_subset/vocal_synthetic_003-053-100.wav', './nsynth_subset/guitar_acoustic_015-085-100.wav', './nsynth_subset/guitar_electronic_028-054-127.wav', './nsynth_subset/vocal_synthetic_003-068-075.wav', './nsynth_subset/guitar_acoustic_014-106-075.wav', './nsynth_subset/vocal_synthetic_003-049-025.wav', './nsynth_subset/vocal_synthetic_003-061-100.wav', './nsynth_subset/vocal_ac

### Install and Load Required Libraries  

In [3]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [5]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [6]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [7]:
# Some helper functions. These are the same as what the autograder runs.

In [8]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [9]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [10]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [11]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [12]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [13]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [14]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 20, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [15]:
def test2(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [16]:
test2(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.003,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.8085838490062289, validation accuracy = 0.7154471544715447
Epoch 1, loss = 0.24131055445306832, validation accuracy = 0.7560975609756098
Epoch 2, loss = 0.1293597628052036, validation accuracy = 0.4796747967479675
Epoch 3, loss = 0.08542136651360327, validation accuracy = 0.4796747967479675
Epoch 4, loss = 0.055580574838030666, validation accuracy = 0.4796747967479675
Epoch 5, loss = 0.03855626753324436, validation accuracy = 1.0
Epoch 6, loss = 0.029594562883074913, validation accuracy = 1.0
Epoch 7, loss = 0.02510577128527479, validation accuracy = 1.0
Epoch 8, loss = 0.020592547125286527, validation accuracy = 1.0
Epoch 9, loss = 0.014905108999098755, validation accuracy = 0.983739837398374
Final validation accuracy = 0.983739837398374, best = 1.0
Test accuracy = 1.0
